In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
fpt_data=pd.read_csv('/content/drive/MyDrive/Barefoots/Vòng 2/Code/Main/PreTrain Data/FPT raw.csv')
cmc_data=pd.read_csv('/content/drive/MyDrive/Barefoots/Vòng 2/Code/Main/PreTrain Data/CMC raw.csv')


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
fpt_data1=fpt_data.copy()
fpt_data1.drop('Date', axis=1, inplace=True)
# Tạo các target variables cho các khoảng thời gian dự báo khác nhau
forecast_days = [3, 7, 21, 63]
for day in forecast_days:
    fpt_data1[f'Close_future_{day}d'] = fpt_data1['Close_FPT'].shift(-day)

results = {}

for day in forecast_days:

    fpt_data_temp = fpt_data1.copy()

    # Loại bỏ các cột target đã tạo
    cols_to_drop = [f'Close_future_{d}d' for d in forecast_days if f'Close_future_{d}d' in fpt_data_temp.columns]

    # Giữ lại tất cả các cột features để dự đoán
    X_columns = [col for col in fpt_data_temp.columns if col not in cols_to_drop and (col != f'Close_future_{day}d' and col !='Close_FPT')]

    # Chuẩn bị dữ liệu
    df_model = fpt_data_temp[[*X_columns, f'Close_future_{day}d']].dropna()

    X = df_model[X_columns]
    y = df_model[f'Close_future_{day}d']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Huấn luyện mô hình
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Tính các chỉ số đánh giá
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Lưu kết quả
    results[day] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}


# In kết quả chi tiết
for day, metrics in results.items():
    print(f"\nDự đoán {day} ngày:")
    print(f"   MAE : {metrics['MAE']:.4f}")
    print(f"   RMSE: {metrics['RMSE']:.4f}")
    print(f"   R²   : {metrics['R2']:.4f}")



Dự đoán 3 ngày:
   MAE : 11.4163
   RMSE: 14.7226
   R²   : 0.6494

Dự đoán 7 ngày:
   MAE : 11.9460
   RMSE: 15.4657
   R²   : 0.6122

Dự đoán 21 ngày:
   MAE : 8.3620
   RMSE: 10.5235
   R²   : 0.8188

Dự đoán 63 ngày:
   MAE : 12.7656
   RMSE: 14.8795
   R²   : 0.6300


In [ ]:
fpt_data.head()

,Date,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital,Fixed Asset Turnover,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,EBIT Margin (%),...,ADX_63_FPT,ADX_100_FPT,TRIX_mid_term21_FPT,TRIX_mid_term63_FPT,TRIX_long_term100_FPT,TSF_3_FPT,TSF_7_FPT,TSF_21_FPT,TSF_63_FPT,TSF_100_FPT
0,2017-08-24,1.308254,0.390988,0.825945,9.086943,52.127014,35.171836,131.337346,7.002128,0.079915,...,0.739749,0.271649,0.000133,0.000141,0.000212,14.560000,14.907143,14.992762,16.508233,18.414485
1,2017-08-25,1.308254,0.390988,0.825945,9.086943,52.127014,35.171836,131.337346,7.002128,0.079915,...,0.739749,0.271649,0.000133,0.000141,0.000212,14.560000,14.907143,14.992762,16.508233,18.414485
2,2017-08-28,1.308254,0.390988,0.825945,9.086943,52.127014,35.171836,131.337346,7.002128,0.079915,...,0.739749,0.271649,0.000133,0.000141,0.000212,14.560000,14.907143,14.992762,16.508233,18.414485
3,2017-08-29,1.308254,0.390988,0.825945,9.086943,52.127014,35.171836,131.337346,7.002128,0.079915,...,0.739749,0.271649,0.000133,0.000141,0.000212,14.560000,14.907143,14.992762,16.508233,18.414485
4,2017-08-30,1.308254,0.390988,0.825945,9.086943,52.127014,35.171836,131.337346,7.002128,0.079915,...,0.739749,0.271649,0.000133,0.000141,0.000212,14.676667,14.907143,14.992762,16.508233,18.414485


In [ ]:
last_date = pd.to_datetime(fpt_data['Date'].iloc[-1])

future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 8)]

# Get the last row of fpt_data1 (excluding 'Close_FPT' and target columns)
last_row = fpt_data1.drop(columns=['Close_FPT'] + [f'Close_future_{day}d' for day in forecast_days]).iloc[-1]

# Create X_future with all the features used during training
X_future = pd.DataFrame([last_row.values], columns=last_row.index)

# Repeat the last row for the desired number of future predictions
X_future = pd.concat([X_future] * 7, ignore_index=True)

# Now you can predict:
y_future = model.predict(X_future)

future_df = pd.DataFrame({
    'Date': future_dates,
    'Close_Price_Prediction': y_future
})

future_df

,Date,Close_Price_Prediction
0,2025-04-10,110.428593
1,2025-04-11,110.428593
2,2025-04-12,110.428593
3,2025-04-13,110.428593
4,2025-04-14,110.428593
5,2025-04-15,110.428593
6,2025-04-16,110.428593


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Giả sử fpt_data đã được load và có cột 'Close_FPT' + các feature khác + 'Date'
# Tạo các target variables
forecast_days = [3, 7, 21, 63]
for day in forecast_days:
    fpt_data[f'Close_future_{day}d'] = fpt_data['Close_FPT'].shift(-day)

results = {}

for day in forecast_days:
    print(f"\n>>> Xử lý dự đoán {day} ngày <<<")

    # Chuẩn bị DataFrame cho task này
    # Giữ cột Date, shift, các feature và target tương ứng
    cols_to_drop = [f'Close_future_{d}d' for d in forecast_days if d != day]
    X_cols = [c for c in fpt_data.columns
              if c not in cols_to_drop + [f'Close_future_{day}d','Date']]

    df = fpt_data[X_cols + [f'Close_future_{day}d']].dropna()
    X = df[X_cols]
    y = df[f'Close_future_{day}d']

    # Train-test split (dùng sklearn helper luôn)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    # Pipeline: scale → SVR
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR())
    ])

    # Tham số grid (rút gọn)
    param_grid = {
        'svr__C': [0,1,1, 10, 100],
        'svr__gamma': ['scale',0,1, 0.01],
        'svr__kernel': ['rbf']
    }

    # GridSearch
    grid = GridSearchCV(
        pipe, param_grid,
        cv=5,                 # ít fold cho nhanh
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=0             # tắt log chi tiết
    )
    grid.fit(X_train, y_train)

    best = grid.best_estimator_
    print(" • Best params:", grid.best_params_)

    # Dự đoán & đánh giá
    y_pred = best.predict(X_test)
    mae  = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2   = r2_score(y_test, y_pred)

    results[day] = {
        'MAE': mae, 'RMSE': rmse, 'R2': r2,
        'best_params': grid.best_params_
    }

    print(f" • MAE : {mae:.4f}")
    print(f" • RMSE: {rmse:.4f}")
    print(f" • R²  : {r2:.4f}")

# In ra bảng tổng kết
summary = pd.DataFrame([
    {
      'Horizon (ngày)': d,
      'MAE': f"{v['MAE']:.4f}",
      'RMSE': f"{v['RMSE']:.4f}",
      'R2': f"{v['R2']:.4f}",
      **v['best_params']
    }
    for d, v in results.items()
])
print("\n===== SUMMARY =====")
print(summary.to_string(index=False))



>>> Xử lý dự đoán 3 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 1, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 68.0729
 • RMSE: 76.0082
 • R²  : -9.2154

>>> Xử lý dự đoán 7 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 69.2549
 • RMSE: 76.6878
 • R²  : -9.4725

>>> Xử lý dự đoán 21 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 68.7887
 • RMSE: 76.4967
 • R²  : -9.6185

>>> Xử lý dự đoán 63 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 67.3837
 • RMSE: 74.8636
 • R²  : -9.6358

===== SUMMARY =====
 Horizon (ngày)     MAE    RMSE      R2  svr__C svr__gamma svr__kernel
              3 68.0729 76.0082 -9.2154       1      scale         rbf
              7 69.2549 76.6878 -9.4725     100      scale         rbf
             21 68.7887 76.4967 -9.6185     100      scale         rbf
             63 67.3837 74.8636 -9.6358     100      scale         rbf


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
cmc_data1=cmc_data.copy()
cmc_data1.drop('Date', axis=1, inplace=True)
# Tạo các target variables cho các khoảng thời gian dự báo khác nhau
forecast_days = [3, 7, 21, 63]
for day in forecast_days:
    cmc_data1[f'Close_future_{day}d'] = cmc_data1['Close_CMC'].shift(-day)

results = {}

for day in forecast_days:

    cmc_data_temp = cmc_data1.copy()

    # Loại bỏ các cột target đã tạo
    cols_to_drop = [f'Close_future_{d}d' for d in forecast_days if f'Close_future_{d}d' in cmc_data_temp.columns]

    # Giữ lại tất cả các cột features để dự đoán
    X_columns = [col for col in cmc_data_temp.columns if col not in cols_to_drop and (col != f'Close_future_{day}d' and col !='Close_cmc')]

    # Chuẩn bị dữ liệu
    df_model = cmc_data_temp[[*X_columns, f'Close_future_{day}d']].dropna()

    X = df_model[X_columns]
    y = df_model[f'Close_future_{day}d']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Huấn luyện mô hình
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Tính các chỉ số đánh giá
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Lưu kết quả
    results[day] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}


# In kết quả chi tiết
for day, metrics in results.items():
    print(f"\nDự đoán {day} ngày:")
    print(f"   MAE : {metrics['MAE']:.4f}")
    print(f"   RMSE: {metrics['RMSE']:.4f}")
    print(f"   R²   : {metrics['R2']:.4f}")



Dự đoán 3 ngày:
   MAE : 5.2742
   RMSE: 7.4418
   R²   : 0.1115

Dự đoán 7 ngày:
   MAE : 15.7088
   RMSE: 21.9627
   R²   : -6.7511

Dự đoán 21 ngày:
   MAE : 46.6822
   RMSE: 65.8937
   R²   : -69.1743

Dự đoán 63 ngày:
   MAE : 16.3395
   RMSE: 24.7065
   R²   : -8.9859


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


forecast_days = [3, 7, 21, 63]
for day in forecast_days:
    cmc_data[f'Close_future_{day}d'] = cmc_data['Close_CMC'].shift(-day)

results = {}

for day in forecast_days:
    print(f"\n>>> Xử lý dự đoán {day} ngày <<<")

    # Chuẩn bị DataFrame cho task này
    # Giữ cột Date, shift, các feature và target tương ứng
    cols_to_drop = [f'Close_future_{d}d' for d in forecast_days if d != day]
    X_cols = [c for c in cmc_data.columns
              if c not in cols_to_drop + [f'Close_future_{day}d','Date']]

    df = cmc_data[X_cols + [f'Close_future_{day}d']].dropna()
    X = df[X_cols]
    y = df[f'Close_future_{day}d']

    # Train-test split (dùng sklearn helper luôn)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    # Pipeline: scale → SVR
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR())
    ])

    # Tham số grid (rút gọn)
    param_grid = {
        'svr__C': [0,1,1, 10, 100],
        'svr__gamma': ['scale',0,1, 0.01],
        'svr__kernel': ['rbf']
    }

    # GridSearch
    grid = GridSearchCV(
        pipe, param_grid,
        cv=5,                 # ít fold cho nhanh
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=0             # tắt log chi tiết
    )
    grid.fit(X_train, y_train)

    best = grid.best_estimator_
    print(" • Best params:", grid.best_params_)

    # Dự đoán & đánh giá
    y_pred = best.predict(X_test)
    mae  = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2   = r2_score(y_test, y_pred)

    results[day] = {
        'MAE': mae, 'RMSE': rmse, 'R2': r2,
        'best_params': grid.best_params_
    }

    print(f" • MAE : {mae:.4f}")
    print(f" • RMSE: {rmse:.4f}")
    print(f" • R²  : {r2:.4f}")

# In ra bảng tổng kết
summary = pd.DataFrame([
    {
      'Horizon (ngày)': d,
      'MAE': f"{v['MAE']:.4f}",
      'RMSE': f"{v['RMSE']:.4f}",
      'R2': f"{v['R2']:.4f}",
      **v['best_params']
    }
    for d, v in results.items()
])
print("\n===== SUMMARY =====")
print(summary.to_string(index=False))



>>> Xử lý dự đoán 3 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 1, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 20.6435
 • RMSE: 24.4640
 • R²  : -8.6019

>>> Xử lý dự đoán 7 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 20.7936
 • RMSE: 24.5238
 • R²  : -8.6641

>>> Xử lý dự đoán 21 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 20.3850
 • RMSE: 23.7840
 • R²  : -8.1424

>>> Xử lý dự đoán 63 ngày <<<


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

 • Best params: {'svr__C': 10, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
 • MAE : 18.6970
 • RMSE: 22.0092
 • R²  : -6.9245

===== SUMMARY =====
 Horizon (ngày)     MAE    RMSE      R2  svr__C svr__gamma svr__kernel
              3 20.6435 24.4640 -8.6019       1      scale         rbf
              7 20.7936 24.5238 -8.6641     100      scale         rbf
             21 20.3850 23.7840 -8.1424     100      scale         rbf
             63 18.6970 22.0092 -6.9245      10      scale         rbf
